## Gen by IICONTENT
##### Author - Akash Shetty

This notebook shows the implementation of the following features -

1. Gemini Initialization
2. Function Calling with Gemini
3. Integration </br>
    i. Google Docs </br>
    ii. Google Speech to Text </br>
4. Create Jira ticket via Gemini Function Calling

### Gemini Initialization

Here I initialize Gemini with Vertex AI and test where gemini chat works.

In [4]:
import vertexai

PROJECT_ID = "project_id" # Replace with your project ID
LOCATION = 'us-central1'
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [9]:
from vertexai.generative_models import GenerativeModel, ChatSession 

model = GenerativeModel(
    "gemini-1.0-pro-001")

chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

prompt = "Hello."
print(get_chat_response(chat, prompt))

prompt = "What are all the colors in a rainbow?"
print(get_chat_response(chat, prompt))

Hello there. How are you doing today?
The traditional order of the colors in a rainbow is:

1. Red
2. Orange
3. Yellow
4. Green
5. Blue
6. Indigo
7. Violet

However, some people also include pink and turquoise in their lists of rainbow colors.


### Function Calling with Gemini

Here I test using gemini to use function as api. The function used is to square give number.


In [10]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [11]:
square_number = FunctionDeclaration(
    name = "square_number",
    description = "Square any given number",
    parameters={
        "type": "object",
        "properties": {
            "number": {"type": "string", "description": "The number to square"},
        },
        },
)

square_tool = Tool(
    function_declarations=[square_number],
)

def square_number(params):
    """
    Calculates the square of a given number.

    Args:
        params (dict): A dictionary containing the 'number' parameter.

    Returns:
        int: The square of the given number.
    """
    number = int(params['number'])
    return number ** 2


In [15]:
model = GenerativeModel(
    "gemini-1.0-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[square_tool],
)
chat = model.start_chat()

prompt = """
Square number 5
"""

response = chat.send_message(prompt)
print("""\n
      Output from JSON Extraction
      \n""")
print(response.candidates[0].content.parts[0])

params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key] = value

print("""\n
      Output from params
      \n""")
print(params)

response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Square of given number, explain reasoning"""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "square_number"
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="square_number",
            response={
                "content": square_number(params)
            }
        )
    ]),
    ],
    tools=[square_tool],
)

print("""\n
      Output from function calling
      \n""")
print(response.candidates[0].content.parts[0].text)



      Output from JSON Extraction
      

function_call {
  name: "square_number"
  args {
    fields {
      key: "number"
      value {
        string_value: "5"
      }
    }
  }
}



      Output from params
      

{'number': '5'}


      Output from function calling
      

The square of 5 is 25. This is because when you multiply a number by itself, you are essentially adding it to itself that many times. For example, 5 squared is 5 * 5, which is 25.


### Integration 


#### Google Docs -
Here I show to extract data from google doc. And use Gemini to summarize document. You need oauth2 for google docs api.

In [28]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


SCOPES = ['https://www.googleapis.com/auth/documents.readonly']

def auth_and_get_service():
    creds = None
    try:
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    except:
        pass
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            flow.run_local_server(prompt='select_account')
            creds = flow.credentials
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('docs', 'v1', credentials=creds)
    return service

def get_document_text(document_id):
    service = auth_and_get_service()
    document = service.documents().get(documentId=document_id).execute()
    return document

In [37]:
d = get_document_text("1Nkq5r7WTT0kQ1rDq0UlYOjR_s8rLxCpmWuk3WNhtPko")

In [33]:
def extract_text_from_content(content):
    text = ""
    # Access the body and content safely using get, with default empty values where needed
    for element in content.get('body', {}).get('content', []):
        if 'paragraph' in element:  # Check for paragraph element
            for elem in element['paragraph'].get('elements', []):  # Safely access paragraph elements
                if 'textRun' in elem:
                    text += elem['textRun'].get('content', '')  # Correctly access the text content and handle cases where content might be missing
    return text

print(extract_text_from_content(d))


Project Planning Document
1. Project Overview
Project Title: Development of an AI-Based Recommendation System for E-Commerce
Description: This project aims to design and implement an artificial intelligence (AI) powered recommendation system for an e-commerce website. The system will leverage machine learning algorithms to analyze user behavior and product data to provide personalized product recommendations, thereby enhancing user experience and boosting sales.
2. Project Objectives
To increase customer engagement by providing personalized product recommendations.
To improve the conversion rate and average order value through targeted recommendations.
To enhance the user interface with seamless integration of recommendation features.
3. Project Scope
Data collection and preprocessing from existing e-commerce platforms.
Development of machine learning models for recommendation.
Integration of the recommendation system into the current e-commerce platform.
Testing and optimization base

In [38]:
model = GenerativeModel(
    "gemini-1.0-pro-001")

chat = model.start_chat(response_validation=False)

prompt = "Summarize the document for jira ticket with Acceptance Criteria" + extract_text_from_content(d) 
print(get_chat_response(chat, prompt))

**Jira Ticket: Development of an AI Recommendation System for E-Commerce**

**Acceptance Criteria:**

* **Phase 1: Requirement Analysis and Planning**
    * Project scope, objectives, and requirements defined
    * Feasibility study and resource allocation performed

* **Phase 2: Data Preparation**
    * User interaction data and product catalogs collected and preprocessed
    * Continuous data pipelines established

* **Phase 3: Model Development**
    * ML algorithms developed and trained for recommendation
    * Model performance evaluated and iterated for accuracy

* **Phase 4: System Integration and Testing**
    * Recommendation engine integrated into e-commerce platform
    * Functionality and performance thoroughly tested

* **Phase 5: Launch and Optimization**
    * Recommendation system launched publicly
    * System performance and user feedback monitored
    * Recommendation algorithms optimized based on data and marketing insights

* **Phase 6: Maintenance and Upgrades**
 

### Google Speech to Text
This module was used for convert speech to text, and then the text_blob is summarized using gemini.

In [39]:
import base64
import subprocess
def m4a_to_base64(m4a_path):
    # Convert M4A to a compatible WAV format using ffmpeg
    command = ['ffmpeg', '-i', m4a_path, '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '1', '-f', 'wav', 'pipe:1']
    wav_data = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Encode the WAV binary data to base64
    base64_encoded_data = base64.b64encode(wav_data.stdout)
    return base64_encoded_data.decode('utf-8')

from google.cloud import speech

# Instantiates a client
client = speech.SpeechClient()

# The content of the audio file to transcribe
audio_content = m4a_to_base64("test_audio.m4a")

# transcribe speech
audio = speech.RecognitionAudio(content=audio_content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code="en-US",
    model="default",
    audio_channel_count=1,
    enable_word_confidence=True,
    enable_word_time_offsets=True,
)

# Detects speech in the audio file
operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=90)

for result in response.results:
  print("Transcript: {}".format(result.alternatives[0].transcript))

Waiting for operation to complete...
Transcript: hi I'm my name is Akash I'm working on the Google hackathon for this recording this audio basically I want to create a project for recommendation system this has to be work by the machine learning team
Transcript:  I will be working with the front end developers and back in the flippers on this project


In [40]:
model = GenerativeModel(
    "gemini-1.0-pro-001")

chat = model.start_chat(response_validation=False)

prompt = "Summarize the text for jira ticket with Acceptance Criteria" + result.alternatives[0].transcript
print(get_chat_response(chat, prompt))

**Summary:**

I will collaborate with frontend and backend developers on a project involving flippers.

**Acceptance Criteria:**

* [ ] Define the technical requirements with the frontend developers and back in the flippers.
* [ ] Implement the frontend and backend components as per the agreed requirements.
* [ ] Ensure seamless integration and communication between both teams throughout the project.
* [ ] Deliver a fully functional flipper solution that meets the specifications.


### Jira Integration

This can be created using the Jira API. The API documentation can be found at https://developer.atlassian.com/cloud/jira/platform/rest/v3/api-group-issues/#api-rest-api-3-issue-post.


Initiate the Gemini Jira function and call `gemini_jira(prompt)`
    

In [41]:
# Code not provided in the notebook
def create_jira_ticket(ticket_details):
    """
    Creates a Jira ticket with the provided ticket details.

    Args:
        ticket_details (dict): A dictionary containing the ticket details.
            The dictionary should have the following keys:
            - project_key (str): The key of the project where the ticket will be created.
            - summary (str): The summary of the ticket.
            - issuetype (str): The type of the issue.
            - assignee (str): The ID of the assignee.
            - reporter (str): The ID of the reporter.
            - priority (str): The priority of the ticket.
            - labels (list): A list of labels for the ticket.
            - due_date (str): The due date of the ticket.
            - start_date (str): The start date of the ticket.
            - category (str): The category of the ticket.
            - description (str): The description of the ticket.

    Returns:
        dict: A dictionary containing the response from the Jira API if the ticket is created successfully.

    Raises:
        HTTPError: If there is an error while creating the ticket.
    """
    raise NotImplementedError

In [42]:
jira_create_issue = FunctionDeclaration(
    name = "jira_create_issue",
    description = "Create a new Jira issue",
    parameters={
        "type": "object",
        "properties": {
            "summary": {"type": "string", "description": "Summary of the issue"},
            "priority_id": {"type": "string", "description": "ID of the priority level, With 1 as Highest and 5 as Lowest"},
            "labels": {"type": "array", "description": "List of labels associated with the issue"},
            "due_date": {"type": "string", "description": "Due date of the issue"},
            "start_date": {"type": "string", "description": "Start date of the issue"},
            "description": {"type": "string", "description": "Description of the issue"},
        },
        },
)

In [43]:
jira_tool = Tool(
    function_declarations=[jira_create_issue],
)

In [44]:
model = GenerativeModel(
    "gemini-1.0-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[jira_tool],
)
chat = model.start_chat()

In [45]:
prompt = """
Subject: Urgent: Website Crash on Checkout

Hi Team,

I noticed this morning that the checkout process on our website is crashing when users select the express shipping option. This is critical as it affects all users trying to place orders with express delivery.

Please address this issue as a priority. The error message displayed is "502 Bad Gateway". 
With Due Date: 2024-05-01
Label - Front-End

Best,
John
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "jira_create_issue"
  args {
    fields {
      key: "summary"
      value {
        string_value: "Website Crash on Checkout"
      }
    }
    fields {
      key: "priority_id"
      value {
        string_value: "1"
      }
    }
    fields {
      key: "labels"
      value {
        list_value {
          values {
            string_value: "Front-End"
          }
        }
      }
    }
    fields {
      key: "due_date"
      value {
        string_value: "2024-05-01"
      }
    }
    fields {
      key: "description"
      value {
        string_value: "The checkout process on our website is crashing when users select the express shipping option. The error message displayed is \'502 Bad Gateway\'."
      }
    }
  }
}

In [46]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key] = value

params

{'priority_id': '1',
 'due_date': '2024-05-01',
 'description': "The checkout process on our website is crashing when users select the express shipping option. The error message displayed is '502 Bad Gateway'.",
 'summary': 'Website Crash on Checkout',
 'labels': ['Front-End']}

In [47]:
api_response = {
    "id": "10018",
    "key": "GEN-15",
    "self": "https://placeholder.atlassian.net/rest/api/3/issue/10018"
}

In [51]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Describe Ticket with Acceptance Criteria for Front End, explain reasoning"""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "jira_create_issue"
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="jira_create_issue",
            response={
                "content": api_response
            }
        )
    ]),
    ],
    tools=[jira_tool],
)

print(response.candidates[0].content.parts[0].text)

OK. I have created a ticket for this issue with the ID GEN-15. 

**Description**: The checkout process on our website is crashing when users select the express shipping option. This affects all users trying to place orders with express delivery. The error message displayed is "502 Bad Gateway".

**Acceptance Criteria**:
- The checkout process should not crash when users select the express shipping option.
- The error message "502 Bad Gateway" should not be displayed.

**Priority**: High
**Due Date**: 2024-05-01
**Labels**: Front-End

I have assigned this ticket to the Front-End team. Please let me know if you have any questions.


In [54]:
def gemini_jira(prompt):
    model = GenerativeModel(
        "gemini-1.0-pro-001",
        generation_config=GenerationConfig(temperature=0),
        tools=[jira_tool],
    )
    chat = model.start_chat()
    response = chat.send_message(prompt)
    params = {}
    for key, value in response.candidates[0].content.parts[0].function_call.args.items():
        params[key] = value
    response = model.generate_content(
        [
        Content(role="user", parts=[
            Part.from_text(prompt + """Describe Ticket with Acceptance Criteria for Front End, explain reasoning"""),
        ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call": {
                    "name": "jira_create_issue"
                }
            })
        ]),
        Content(role="function", parts=[
            Part.from_function_response(
                name="jira_create_issue",
                response={
                    "content": api_response
                }
            )
        ]),
        ],
        tools=[jira_tool],
    )

    return response.candidates[0].content.parts[0].text

In [56]:
prompt = """
Subject: Urgent: Website Crash on Checkout

Hi Team,

I noticed this morning that the checkout process on our website is crashing when users select the express shipping option. This is critical as it affects all users trying to place orders with express delivery.

Please address this issue as a priority. The error message displayed is "502 Bad Gateway". 
With Due Date: 2024-05-01
Label - Front-End

Best,
John
"""
print(gemini_jira(prompt))

OK. I have created a ticket for this issue with the ID GEN-15. 

**Description**: The checkout process on our website is crashing when users select the express shipping option. This affects all users trying to place orders with express delivery. The error message displayed is "502 Bad Gateway".

**Acceptance Criteria**:
- The checkout process should not crash when users select the express shipping option.
- The error message "502 Bad Gateway" should not be displayed.

**Priority**: High
**Due Date**: 2024-05-01
**Labels**: Front-End

I have assigned this ticket to the Front-End team. Please let me know if you have any questions.
